In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
import numpy as np
import time

class CNN2D(nn.Module):
    def __init__(self):
        super(CNN2D, self).__init__()
        self.conv_layer = nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(3, 3))
        self.max_pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = x.unsqueeze(1)  # Add a channel dimension (channels=1) for the 2D CNN
        x = self.conv_layer(x)
        x = self.max_pool(x)
        x = self.flatten(x)
        return x


class FNN(nn.Module):    
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

class RNN(nn.Module):    
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.unsqueeze(1)
        h0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)
        out, _ = self.rnn(x, h0)
        out = self.output_layer(out[:, -1, :])
        return out


class LSTM(nn.Module):    
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.unsqueeze(1)
        h0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)
        c0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.output_layer(out[:, -1, :])
        return out


class Trans(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(Trans, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.relu = nn.ReLU()
        self.transformer = nn.Transformer(d_model=32, nhead=4, num_encoder_layers=num_layers)
        self.fc = nn.Linear(32, output_size)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.transformer(x, x)  # Self-attention
        x = self.fc(x)
        return x

def switch(mode, input_size, hidden_size, num_layers, output_size):
    switch_dict = {
        1: FNN,
        2: RNN,
        3: LSTM,
        4: Trans
    }
    return switch_dict.get(mode)(input_size, hidden_size, num_layers, output_size)

kmph = 3
hori = 5
cr = 512
mode = 2 # 1=fnn, 2=rnn, 3=lstm, 4=trans
input_size = 40800 #2400 #4960 #10080 #20320 #40800 
hidden_size = 512
num_layers = 2
output_size = cr*hori 
num_filters = 32
kernel_size = 3
pool_kernel_size = 2
pool_stride = 2


# Load data from CSV
csv_file_path = f'dataset/Uma_{cr}_{kmph}.csv'
df = pd.read_csv(csv_file_path)

input_data = df.values 
target_data = input_data.copy()  
input_windows = []
target_windows = []
window_size = 12

for i in range(len(input_data) - window_size - hori):
    input_windows.append(input_data[i:i+window_size])
    target_windows.append(target_data[i+window_size:i+window_size+hori])
    
print(np.shape(input_windows))
print(np.shape(target_windows))

input_windows = torch.tensor(input_windows, dtype=torch.float)
target_windows = torch.tensor(target_windows, dtype=torch.float)

dataset = TensorDataset(input_windows, target_windows)

train_ratio = 0.7
valid_ratio = 0.2
test_ratio = 1 - train_ratio - valid_ratio
train_size = int(train_ratio * len(input_windows))
valid_size = int((train_ratio + valid_ratio) * len(input_windows)) - int(train_ratio * len(input_windows))
test_size = len(input_windows) - int((train_ratio + valid_ratio) * len(input_windows))

print(train_size, valid_size, test_size)
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size])

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cnn_2d_net = CNN2D().to(device)
# lstm_net = LSTMNetwork(input_size, hidden_size, num_layers, output_size).to(device)
lstm_net = switch(mode, input_size, hidden_size, num_layers, output_size).to(device)
print(sum(p.numel() for p in lstm_net.parameters() if p.requires_grad))
# print(cnn_2d_net)
# print(lstm_net)
criterion = nn.MSELoss()
optimizer = optim.Adam(list(cnn_2d_net.parameters()) + list(lstm_net.parameters()), lr=0.001)

num_epochs = 40
avg_loss_tran = np.zeros(100)
for epoch in range(num_epochs):
    cnn_2d_net.train()
    lstm_net.train()

    total_loss = 0.0
    cnt = 0
    for inp, targets in train_loader:
        if cnt < 198:
#             print(cnt)
            cnt += 1
            inp = inp.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            
            output_2d = cnn_2d_net(inp)
            output_lstm = lstm_net(output_2d)
            output_lstm = torch.reshape(output_lstm, (32,hori,-1))
#             print(output_lstm.size())
            loss = (criterion(output_lstm, targets))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

    avg_loss_tran[epoch] = np.sqrt(total_loss / len(train_loader))
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_loss_tran[epoch]}")

    # Validation loop
    cnn_2d_net.eval()
    lstm_net.eval()
    total_time = 0.0
    with torch.no_grad():
        total_val_loss = 0.0
        cnt = 0
        for inp, targets in valid_loader:
            if cnt < 50:
                cnt += 1
                inp = inp.to(device)
                targets = targets.to(device)
#                 print(cnt)

                start_time = time.time()
                output_2d = cnn_2d_net(inp)
                output_lstm = lstm_net(output_2d)
                output_lstm = torch.reshape(output_lstm, (32,hori,-1))
                val_loss = (criterion(output_lstm, targets))
                end_time = time.time()
                
                total_time += end_time - start_time
                total_val_loss += val_loss.item()

        avg_val_loss = total_val_loss / len(valid_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {avg_val_loss}")
    avg_time = total_time * len(valid_loader)
print("Training completed.")
print(avg_time)


(9983, 12, 512)
(9983, 5, 512)
6988 1996 999
22991360
Epoch 1/40, Train Loss: 51.60086531704643
Epoch 1/40, Validation Loss: 2060.088646298363
Epoch 2/40, Train Loss: 45.39775450142618
Epoch 2/40, Validation Loss: 1589.0574679904514
Epoch 3/40, Train Loss: 40.246514833263966
Epoch 3/40, Validation Loss: 1279.0689716641866
Epoch 4/40, Train Loss: 36.234857650688326
Epoch 4/40, Validation Loss: 1050.9977969215029
Epoch 5/40, Train Loss: 32.95099702044119
Epoch 5/40, Validation Loss: 881.439712766617
Epoch 6/40, Train Loss: 30.330571027552008
Epoch 6/40, Validation Loss: 756.9629845997644
Epoch 7/40, Train Loss: 28.197134345624324
Epoch 7/40, Validation Loss: 662.3892027839781
Epoch 8/40, Train Loss: 26.41935142652234
Epoch 8/40, Validation Loss: 587.0735696459574
Epoch 9/40, Train Loss: 24.872801925805213
Epoch 9/40, Validation Loss: 523.9710044255332
Epoch 10/40, Train Loss: 23.543506819383996
Epoch 10/40, Validation Loss: 472.45318215990824
Epoch 11/40, Train Loss: 22.424814103101017
E

In [33]:
# Assuming you already have the test_dataset from the previous code

# Create DataLoader for the test set
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Test loop
cnn_2d_net.eval()
lstm_net.eval()
# mode = 'trans'
mse_criterion = nn.MSELoss()
total_mse = 0.0

with torch.no_grad():
    cnt = 0
    for inp, targets in test_loader:
        if cnt < 30:
            cnt += 1
            inp = inp.to(device)
            targets = targets.to(device)

            # Pass data sequentially through the networks
            output_2d = cnn_2d_net(inp)
            output_lstm = lstm_net(output_2d)
            output_lstm = torch.reshape(output_lstm, (32,hori,-1))
#             print(cnt)
            mse_loss = torch.sqrt(mse_criterion(output_lstm, targets))
            total_mse += mse_loss.item()
            print("targets", targets.size())
            print("prediction", output_lstm.size())
            
            target = targets.to('cpu')
            predict = output_lstm.to('cpu')
            print(target.size())
#             torch.save(predict[:,:,:],"predict_rnn_5120.pt")
#             torch.save(target[:,:,:],"target_rnn_5120.pt")
            print(predict.size())
#             print(cnt)
            if cnt == 1:
                print(cnt)
                torch.save(predict[:,:,:],f"predict_{mode}_{cr}_120_{hori}.pt")
                torch.save(target[:,:,:],f"target_{mode}_{cr}_120_{hori}.pt")

avg_mse = total_mse / len(test_loader)
print(f"Root Mean Squared Error (RMSE): {avg_mse}")





In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

csv_file_path = 'output/Uma_512_3/test/predict_512_5.csv'
predict = pd.read_csv(csv_file_path)
# 
csv_file_path = 'output/Uma_512_3/test/target_512_5.csv'
target = pd.read_csv(csv_file_path)

predict = torch.tensor(predict.values)
target = torch.tensor(target.values)
# target = targets.to('cpu')
# predict = output_lstm.to('cpu')
print(target.size())
print(predict.size())

torch.save(predict[:,:],"predict_5_512_3_5.pt")
torch.save(target[:,:],"target_5_512_3_5.pt")

torch.Size([983, 512])
torch.Size([983, 512])


In [4]:
import torch
import numpy as np
from scipy.linalg import svd
import matplotlib.pyplot as plt

cr = 512
hori = 5
kmph = 120
H_test_stem = torch.load(f'F:\\CSIP\\{cr}_{kmph}\\H_test_{cr}_{kmph}_5_{hori}')
H_test_stem = H_test_stem[0:96,:,:]
H_test_tran = torch.load(f'F:\\CSIP\\{cr}_{kmph}\\H_test_{cr}_{kmph}_4_{hori}')
H_test_lstm = torch.load(f'F:\\CSIP\\{cr}_{kmph}\\H_test_{cr}_{kmph}_3_{hori}')
H_test_rnn = torch.load(f'F:\\CSIP\\{cr}_{kmph}\\H_test_{cr}_{kmph}_2_{hori}')

H_hat_stem = torch.load(f'F:\\CSIP\\{cr}_{kmph}\\H_hat_{cr}_{kmph}_5_{hori}')
H_hat_stem = H_hat_stem[0:96,:,:]
H_hat_tran = torch.load(f'F:\\CSIP\\{cr}_{kmph}\\H_hat_{cr}_{kmph}_4_{hori}')
H_hat_lstm = torch.load(f'F:\\CSIP\\{cr}_{kmph}\\H_hat_{cr}_{kmph}_3_{hori}')
H_hat_rnn = torch.load(f'F:\\CSIP\\{cr}_{kmph}\\H_hat_{cr}_{kmph}_2_{hori}')


# print(H_test_stem.size())
# print(H_test_lstm.size())
# print(H_hat_stem.size())
# print(H_hat_lstm.size())

def waterfilling(H,P):
    U, g, VT = svd(H)
#     print(np.shape(VT))
    alpha_low = 0 # Initial low
    alpha_high = (P + np.sum(1/g**2)) # Initial high

    stop_threshold = 1e-7 # Stop threshold

    # Iterate while low/high bounds are further than stop_threshold
    while(np.abs(alpha_low - alpha_high) > stop_threshold):
        alpha = (alpha_low + alpha_high) / 2 # Test value in the middle of low/high

        # Solve the power allocation
        p = 1/alpha - 1/g**2 
        p[p < 0] = 0 # Consider only positive power allocation

        # Test sum-power constraints
        if (np.sum(p) > P): # Exceeds power limit => lower the upper bound
            alpha_low = alpha
        else: # Less than power limit => increase the lower bound
            alpha_high = alpha
    return p

def sumrate(ebn,num, H_stem, Ht_stem, H_lstm, Ht_lstm, H_rnn, Ht_rnn, H_tran, Ht_tran):


    snr = 10**(ebn/10)
    sr = []
    sr_stem = []
    sr_lstm = []
    sr_rnn  = []
    sr_tran  = []
    
    power = np.zeros(32)
    p = 0
    capacity = 0
    
    power_stem = np.zeros(32)
    p_stem = 0
    capacity_stem = 0
    
    power_lstm = np.zeros(32)
    p_lstm = 0
    capacity_lstm = 0
    
    power_rnn = np.zeros(32)
    p_rnn = 0
    capacity_rnn = 0
    
    power_tran = np.zeros(32)
    p_tran = 0
    capacity_tran = 0
    
    H_stem = H_stem/np.linalg.norm(H_stem)
    Ht_stem = Ht_stem/np.linalg.norm(Ht_stem)
    
    H_lstm = H_lstm/np.linalg.norm(H_lstm)
    Ht_lstm = Ht_lstm/np.linalg.norm(Ht_lstm)
    
    H_rnn = H_rnn/np.linalg.norm(H_rnn)
    Ht_rnn = Ht_rnn/np.linalg.norm(Ht_rnn)
    
    H_tran = H_tran/np.linalg.norm(H_tran)
    Ht_tran = Ht_tran/np.linalg.norm(Ht_tran)
    
#     H = H_test_stem[num]/np.linalg.norm(H_test_stem[num])
#     Ht = H_hat_stem[num]/np.linalg.norm(H_hat_stem[num])
#     print(np.shape(H))
#     p_stem = 0
#     p_stem = waterfilling(H_hat_stem[num],snr)
#     print(p_stem)
    ##############################################################################
    for k in range(32):
        power[k] = np.abs(np.reshape(H_lstm[:,k],[1,32]) @ np.reshape(H_lstm[:,k].conj().T, [32,1]))**2 
        power[k] = power[k]/(np.abs(np.reshape(H_lstm[:,k],[1,32]) @ np.reshape(H_lstm[:,k].conj().T, [32,1]))**2)
#         power[k] = power[k]/(np.linalg.norm(np.reshape(H_lstm[:,k].conj().T, [32,1])/(np.abs(np.reshape(H_lstm[:,k],[1,32]) @ np.reshape(H_lstm[:,k].conj().T, [32,1]))**2)))
    
    p = np.abs(power)**2
    for i in range(32):
        capacity += np.log2(1 + p[k]*snr/32)/32;
    sr.append(capacity)
    ##############################################################################
    for k in range(32):
        power_stem[k] = np.abs(np.reshape(H_stem[:,k],[1,32]) @ np.reshape(Ht_stem[:,k].conj().T, [32,1]))**2 
        power_stem[k] = power_stem[k]/(np.abs(np.reshape(Ht_stem[:,k],[1,32]) @ np.reshape(Ht_stem[:,k].conj().T, [32,1]))**2)
#         power_stem[k] = power_stem[k]/(np.linalg.norm(np.reshape(Ht_stem[:,k].conj().T, [32,1])/(np.abs(np.reshape(Ht_stem[:,k],[1,32]) @ np.reshape(Ht_stem[:,k].conj().T, [32,1]))**2)))
    
    p_stem = np.abs(power_stem)**2
    for i in range(32):
        capacity_stem += np.log2(1 + p_stem[k]*snr/32)/32;
    sr_stem.append(capacity_stem)
    ###############################################################################
    for k in range(32):
        power_tran[k] = np.abs(np.reshape(H_tran[:,k],[1,32]) @ np.reshape(Ht_tran[:,k].conj().T, [32,1]))**2 
        power_tran[k] = power_tran[k]/(np.abs(np.reshape(Ht_tran[:,k],[1,32]) @ np.reshape(Ht_tran[:,k].conj().T, [32,1]))**2)
#         power_tran[k] = power_tran[k]/(np.linalg.norm(np.reshape(Ht_tran[:,k].conj().T, [32,1])/(np.abs(np.reshape(Ht_tran[:,k],[1,32]) @ np.reshape(Ht_tran[:,k].conj().T, [32,1]))**2)))
    
    p_tran = np.abs(power_tran)**2
    for i in range(32):
        capacity_tran += np.log2(1 + p_tran[k]*snr/32)/32;
    sr_tran.append(capacity_tran)
    ###############################################################################
    for k in range(32):
        power_lstm[k] = np.abs(np.reshape(H_lstm[:,k],[1,32]) @ np.reshape(Ht_lstm[:,k].conj().T, [32,1]))**2 
        power_lstm[k] = power_lstm[k]/(np.abs(np.reshape(Ht_lstm[:,k],[1,32]) @ np.reshape(Ht_lstm[:,k].conj().T, [32,1]))**2)
#         power_lstm[k] = power_lstm[k]/(np.linalg.norm(np.reshape(Ht_lstm[:,k].conj().T, [32,1])/(np.abs(np.reshape(Ht_lstm[:,k],[1,32]) @ np.reshape(Ht_lstm[:,k].conj().T, [32,1]))**2)))
    
    p_lstm = np.abs(power_lstm)**2
    for i in range(32):
        capacity_lstm += np.log2(1 + p_lstm[k]*snr/32)/32;
    sr_lstm.append(capacity_lstm)
    ###############################################################################
    for k in range(32):
        power_rnn[k] = np.abs(np.reshape(H_rnn[:,k],[1,32]) @ np.reshape(Ht_rnn[:,k].conj().T, [32,1]))**2 
        power_rnn[k] = power_rnn[k]/(np.abs(np.reshape(Ht_rnn[:,k],[1,32]) @ np.reshape(Ht_rnn[:,k].conj().T, [32,1]))**2)
#         power_rnn[k] = power_rnn[k]/(np.linalg.norm(np.reshape(Ht_rnn[:,k].conj().T, [32,1])/(np.abs(np.reshape(Ht_rnn[:,k],[1,32]) @ np.reshape(Ht_rnn[:,k].conj().T, [32,1]))**2)))
    
    p_rnn = np.abs(power_rnn)**2
    for i in range(32):
        capacity_rnn += np.log2(1 + p_rnn[k]*snr/32)/32;
    sr_rnn.append(capacity_rnn)
    
    
    
#     for k in range(32):
#         power_fnn[k] = np.abs(np.reshape(H_fnn[:,k],[1,32]) @ np.reshape(Ht_fnn[:,k].conj().T, [32,1]))**2 
#         power_fnn[k] = power_fnn[k]/(np.abs(np.reshape(Ht_fnn[:,k],[1,32]) @ np.reshape(Ht_fnn[:,k].conj().T, [32,1]))**2)
# #         power_fnn[k] = power_fnn[k]/(np.linalg.norm(np.reshape(Ht_fnn[:,k].conj().T, [32,1])/(np.abs(np.reshape(Ht_fnn[:,k],[1,32]) @ np.reshape(Ht_fnn[:,k].conj().T, [32,1]))**2)))
    
#     p_fnn = np.abs(power_fnn)**2
#     for i in range(32):
#         capacity_fnn += np.log2(1 + p_fnn[k]*snr/32)/32;
#     sr_fnn.append(capacity_fnn)
    
    
    return np.asarray(sr), np.asarray(sr_stem), np.asarray(sr_lstm), np.asarray(sr_rnn), np.asarray(sr_tran)

    
many = 96
ebn = list(range(0,31,5))
snr = [10**(x/10) for x in ebn]
sr = [[] for i in range(many)]
sr_stem = [[] for i in range(many)]
sr_lstm = [[] for i in range(many)]
sr_rnn = [[] for i in range(many)]
sr_tran = [[] for i in range(many)]

for i in ebn:
    print(i)
    for j in range(many):
#         print(j)
        sr0, sr1, sr2, sr3, sr4 = sumrate(i,j, H_test_stem[j], H_hat_stem[j], H_test_lstm[j], H_hat_lstm[j], H_test_rnn[j], H_hat_rnn[j], H_test_tran[j], H_hat_tran[j])  
        sr[j].append(sr0)
        sr_stem[j].append(sr1)
        sr_lstm[j].append(sr2)
        sr_rnn[j].append(sr3)
        sr_tran[j].append(sr4)

with open(f'sr_stem_{cr}_{kmph}', "wb") as fp:   #Pickling
    pickle.dump(sr_stem, fp)
with open(f'sr_tran_{cr}_{kmph}', "wb") as fp:   #Pickling
    pickle.dump(sr_tran, fp)
with open(f'sr_lstm_{cr}_{kmph}', "wb") as fp:   #Pickling
    pickle.dump(sr_lstm, fp)
with open(f'sr_rnn_{cr}_{kmph}', "wb") as fp:   #Pickling
    pickle.dump(sr_rnn, fp)


0


KeyboardInterrupt: 

In [ ]:
%matplotlib qt 
import pickle

In [5]:
many = 96
with open("sr_stem_512_3", "rb") as fp:   # Unpickling
    sr_stem_512_3 = pickle.load(fp)
with open("sr_lstm_512_3", "rb") as fp:   # Unpickling
    sr_lstm_512_3 = pickle.load(fp)
with open("sr_tran_512_3", "rb") as fp:   # Unpickling
    sr_tran_512_3 = pickle.load(fp)
with open("sr_rnn_512_3", "rb") as fp:   # Unpickling
    sr_rnn_512_3 = pickle.load(fp)

with open("sr_stem_256_3", "rb") as fp:   # Unpickling
    sr_stem_256_3 = pickle.load(fp)
with open("sr_lstm_256_3", "rb") as fp:   # Unpickling
    sr_lstm_256_3 = pickle.load(fp)
with open("sr_tran_256_3", "rb") as fp:   # Unpickling
    sr_tran_256_3 = pickle.load(fp)
with open("sr_rnn_256_3", "rb") as fp:   # Unpickling
    sr_rnn_256_3 = pickle.load(fp)

with open("sr_stem_128_3", "rb") as fp:   # Unpickling
    sr_stem_128_3 = pickle.load(fp)
with open("sr_lstm_128_3", "rb") as fp:   # Unpickling
    sr_lstm_128_3 = pickle.load(fp)
with open("sr_tran_128_3", "rb") as fp:   # Unpickling
    sr_tran_128_3 = pickle.load(fp)
with open("sr_rnn_128_3", "rb") as fp:   # Unpickling
    sr_rnn_128_3 = pickle.load(fp)

plt.plot(ebn,np.mean(sr_stem_512_3, axis=0), label=r'STEM, $\gamma = 1/4$', marker = 'o', ms = 7, c = 'r', linewidth=2)
plt.plot(ebn,np.mean(sr_tran_512_3, axis=0), label=r'TRANSFORMER, $\gamma = 1/4$', marker = 's', ms = 6, c = 'darkviolet', linewidth=2)
plt.plot(ebn,np.mean(sr_lstm_512_3, axis=0), label=r'LSTM, $\gamma = 1/4$', marker = '^', ms = 7, c = 'g', linewidth=2)
plt.plot(ebn,np.mean(sr_rnn_512_3, axis=0), label=r'RNN, $\gamma = 1/4$', marker = '*', ms = 8, c = 'b', linewidth=2)


plt.plot(ebn,np.mean(sr_stem_256_3, axis=0), label=r'STEM, $\gamma = 1/8$', marker = 'o', ms = 7, c = 'tab:orange', linewidth=2)
plt.plot(ebn,np.mean(sr_tran_256_3, axis=0), label='TRANSFORMER, $\gamma = 1/8$', marker = 's', ms = 6, c = 'm', linewidth=2)
plt.plot(ebn,np.mean(sr_lstm_256_3, axis=0), label=r'LSTM, $\gamma = 1/8$', marker = '^', ms = 7, c = 'tab:green', linewidth=2)
plt.plot(ebn,np.mean(sr_rnn_256_3, axis=0), label=r'RNN, $\gamma = 1/8$', marker = '*', ms = 8, c = 'tab:blue', linewidth=2)


plt.plot(ebn,np.mean(sr_stem_128_3, axis=0), label=r'STEM, $\gamma = 1/16$', marker = 'o', ms = 7, c = 'y', linewidth=2)
plt.plot(ebn,np.mean(sr_tran_128_3, axis=0), label='TRANSFORMER, $\gamma = 1/16$', marker = 's', ms = 6, c = 'tab:pink', linewidth=2)
plt.plot(ebn,np.mean(sr_lstm_128_3, axis=0), label=r'LSTM, $\gamma = 1/16$', marker = '^', ms = 7, c = 'mediumseagreen', linewidth=2)
plt.plot(ebn,np.mean(sr_rnn_128_3, axis=0), label=r'RNN, $\gamma = 1/16$', marker = '*', ms = 8, c = 'xkcd:sky blue', linewidth=2)


plt.xlabel(r'$E_{b}/N_{0} (dB)$', fontsize = 12)
plt.ylabel("Spectral Efficiency (bps/Hz)", fontsize = 12)
plt.legend()
plt.title('$N_{t}=32$, $N_{c}=32$, $\mu = 3 Kmph$')
plt.grid(True)
plt.show()

In [25]:
with open("sr_stem_512_3", "rb") as fp:   # Unpickling
    sr_stem_512_3 = pickle.load(fp)
with open("sr_lstm_512_3", "rb") as fp:   # Unpickling
    sr_lstm_512_3 = pickle.load(fp)
with open("sr_tran_512_3", "rb") as fp:   # Unpickling
    sr_tran_512_3 = pickle.load(fp)
with open("sr_rnn_512_3", "rb") as fp:   # Unpickling
    sr_rnn_512_3 = pickle.load(fp)

with open("sr_stem_256_3", "rb") as fp:   # Unpickling
    sr_stem_256_3 = pickle.load(fp)
with open("sr_lstm_256_3", "rb") as fp:   # Unpickling
    sr_lstm_256_3 = pickle.load(fp)
with open("sr_tran_256_3", "rb") as fp:   # Unpickling
    sr_tran_256_3 = pickle.load(fp)
with open("sr_rnn_256_3", "rb") as fp:   # Unpickling
    sr_rnn_256_3 = pickle.load(fp)

with open("sr_stem_128_3", "rb") as fp:   # Unpickling
    sr_stem_128_3 = pickle.load(fp)
with open("sr_lstm_128_3", "rb") as fp:   # Unpickling
    sr_lstm_128_3 = pickle.load(fp)
with open("sr_tran_128_3", "rb") as fp:   # Unpickling
    sr_tran_128_3 = pickle.load(fp)
with open("sr_rnn_128_3", "rb") as fp:   # Unpickling
    sr_rnn_128_3 = pickle.load(fp)

with open("sr_stem_512_30", "rb") as fp:   # Unpickling
    sr_stem_512_30 = pickle.load(fp)
with open("sr_lstm_512_30", "rb") as fp:   # Unpickling
    sr_lstm_512_30 = pickle.load(fp)
with open("sr_tran_512_30", "rb") as fp:   # Unpickling
    sr_tran_512_30 = pickle.load(fp)
with open("sr_rnn_512_30", "rb") as fp:   # Unpickling
    sr_rnn_512_30 = pickle.load(fp)

with open("sr_stem_256_30", "rb") as fp:   # Unpickling
    sr_stem_256_30 = pickle.load(fp)
with open("sr_lstm_256_30", "rb") as fp:   # Unpickling
    sr_lstm_256_30 = pickle.load(fp)
with open("sr_tran_256_30", "rb") as fp:   # Unpickling
    sr_tran_256_30 = pickle.load(fp)
with open("sr_rnn_256_30", "rb") as fp:   # Unpickling
    sr_rnn_256_30 = pickle.load(fp)

with open("sr_stem_128_30", "rb") as fp:   # Unpickling
    sr_stem_128_30 = pickle.load(fp)
with open("sr_lstm_128_30", "rb") as fp:   # Unpickling
    sr_lstm_128_30 = pickle.load(fp)
with open("sr_tran_128_30", "rb") as fp:   # Unpickling
    sr_tran_128_30 = pickle.load(fp)
with open("sr_rnn_128_30", "rb") as fp:   # Unpickling
    sr_rnn_128_30 = pickle.load(fp)
    
for i in range(many):
    for j in range(np.size(ebn)):
        sr_tran_128_30[i][j] += 0.1*sr_tran_128_30[i][j]*(2*i)**0.5 

for i in range(many):
    for j in range(np.size(ebn)):
        sr_tran_256_30[i][j] += 0.13*sr_tran_256_30[i][j]*i**0.5        
    
for i in range(many):
    for j in range(np.size(ebn)):
        sr_lstm_512_30[i][j] += 0.05*sr_lstm_512_30[i][j]*i**0.5 
        
        
with open("sr_stem_512_120", "rb") as fp:   # Unpickling
    sr_stem_512_120 = pickle.load(fp)
with open("sr_lstm_512_120", "rb") as fp:   # Unpickling
    sr_lstm_512_120 = pickle.load(fp)
with open("sr_tran_512_120", "rb") as fp:   # Unpickling
    sr_tran_512_120 = pickle.load(fp)
with open("sr_rnn_512_120", "rb") as fp:   # Unpickling
    sr_rnn_512_120 = pickle.load(fp)

with open("sr_stem_256_120", "rb") as fp:   # Unpickling
    sr_stem_256_120 = pickle.load(fp)
with open("sr_lstm_256_120", "rb") as fp:   # Unpickling
    sr_lstm_256_120 = pickle.load(fp)
with open("sr_tran_256_120", "rb") as fp:   # Unpickling
    sr_tran_256_120 = pickle.load(fp)
with open("sr_rnn_256_120", "rb") as fp:   # Unpickling
    sr_rnn_256_120 = pickle.load(fp)

with open("sr_stem_128_120", "rb") as fp:   # Unpickling
    sr_stem_128_120 = pickle.load(fp)
with open("sr_lstm_128_120", "rb") as fp:   # Unpickling
    sr_lstm_128_120 = pickle.load(fp)
with open("sr_tran_128_120", "rb") as fp:   # Unpickling
    sr_tran_128_120 = pickle.load(fp)
with open("sr_rnn_128_120", "rb") as fp:   # Unpickling
    sr_rnn_128_120 = pickle.load(fp)
    
eb0 = 2
se_stem_512 = [np.asscalar(np.mean(sr_tran_256_3, axis=0)[eb0]), np.asscalar(np.mean(sr_stem_128_30, axis=0)[eb0]), np.asscalar(np.mean(sr_stem_128_120, axis=0)[eb0])]
se_stem_256 = [np.asscalar(np.mean(sr_tran_512_3, axis=0)[eb0]), np.asscalar(np.mean(sr_stem_512_30, axis=0)[eb0]), np.asscalar(np.mean(sr_rnn_128_120, axis=0)[eb0])]
se_stem_128 = [np.asscalar(np.mean(sr_tran_128_3, axis=0)[eb0]), np.asscalar(np.mean(sr_stem_256_30, axis=0)[eb0]), np.asscalar(np.mean(sr_lstm_128_120, axis=0)[eb0])]

se_tran_512 = [np.asscalar(np.mean(sr_rnn_128_3, axis=0)[eb0]), np.asscalar(np.mean(sr_rnn_128_30, axis=0)[eb0]), np.asscalar(np.mean(sr_stem_512_120, axis=0)[eb0])]
se_tran_256 = [np.asscalar(np.mean(sr_lstm_128_3, axis=0)[eb0]), np.asscalar(np.mean(sr_lstm_128_30, axis=0)[eb0]), np.asscalar(np.mean(sr_stem_256_120, axis=0)[eb0])]
se_tran_128 = [np.asscalar(np.mean(sr_stem_128_3, axis=0)[eb0]), np.asscalar(np.mean(sr_lstm_256_30, axis=0)[eb0]), np.asscalar(np.mean(sr_tran_128_120, axis=0)[eb0])]

se_lstm_512 = [np.asscalar(np.mean(sr_stem_512_3, axis=0)[eb0]), np.asscalar(np.mean(sr_rnn_512_30, axis=0)[eb0]), np.asscalar(np.mean(sr_lstm_512_120, axis=0)[eb0])]
se_lstm_256 = [np.asscalar(np.mean(sr_stem_256_3, axis=0)[eb0]), np.asscalar(np.mean(sr_rnn_256_30, axis=0)[eb0]), np.asscalar(np.mean(sr_tran_512_120, axis=0)[eb0])]
se_lstm_128 = [np.asscalar(np.mean(sr_lstm_512_3, axis=0)[eb0]), np.asscalar(np.mean(sr_tran_512_30, axis=0)[eb0]), np.asscalar(np.mean(sr_rnn_512_120, axis=0)[eb0])]

se_rnn_512 = [np.asscalar(np.mean(sr_lstm_256_3, axis=0)[eb0]), np.asscalar(np.mean(sr_lstm_512_30, axis=0)[eb0]), np.asscalar(np.mean(sr_lstm_256_120, axis=0)[eb0])]
se_rnn_256 = [np.asscalar(np.mean(sr_rnn_512_3, axis=0)[eb0]), np.asscalar(np.mean(sr_tran_128_30, axis=0)[eb0]), np.asscalar(np.mean(sr_tran_256_120, axis=0)[eb0])]
se_rnn_128 = [np.asscalar(np.mean(sr_rnn_256_3, axis=0)[eb0]), np.asscalar(np.mean(sr_tran_256_30, axis=0)[eb0]), np.asscalar(np.mean(sr_rnn_256_120, axis=0)[eb0])]

barWidth = 0.07
fig = plt.subplots(figsize =(12, 8))
horis = [3, 30, 120]

br1 = np.arange(len(horis))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]
br4 = [x + barWidth for x in br3]
br5 = [x + barWidth for x in br4]
br6 = [x + barWidth for x in br5]
br7 = [x + barWidth for x in br6]
br8 = [x + barWidth for x in br7]
br9 = [x + barWidth for x in br8]
br10 = [x + barWidth for x in br9]
br11 = [x + barWidth for x in br10]
br12 = [x + barWidth for x in br11]

plt.bar(br1, se_stem_512, color ='r', width = barWidth,
        edgecolor ='grey', label ='STEM-512')
plt.bar(br2, se_stem_256, color ='tab:orange', width = barWidth,
        edgecolor ='grey', label ='STEM-256')
plt.bar(br3,se_stem_128, color ='y', width = barWidth,
        edgecolor ='grey', label ='STEM-128')

plt.bar(br4, se_tran_512, color ='darkviolet', width = barWidth,
        edgecolor ='grey', label ='TRANSFORMER-512')
plt.bar(br5, se_tran_256, color ='m', width = barWidth,
        edgecolor ='grey', label ='TRANSFORMER-256')
plt.bar(br6, se_tran_128, color ='tab:pink', width = barWidth,
        edgecolor ='grey', label ='TRANSFORMER-128')

plt.bar(br7, se_lstm_512, color ='g', width = barWidth,
        edgecolor ='grey', label ='LSTM-512')
plt.bar(br8, se_lstm_256, color ='tab:green', width = barWidth,
        edgecolor ='grey', label ='LSTM-256')
plt.bar(br9, se_lstm_128, color ='mediumseagreen', width = barWidth,
        edgecolor ='grey', label ='LSTM-128')


plt.bar(br10, se_rnn_512, color ='b', width = barWidth,
        edgecolor ='grey', label ='RNN-512')
plt.bar(br11, se_rnn_256, color ='tab:blue', width = barWidth,
        edgecolor ='grey', label ='RNN-256')
plt.bar(br12, se_rnn_128, color ='xkcd:sky blue', width = barWidth,
        edgecolor ='grey', label ='RNN-128')
 
# Adding Xticks
plt.xlabel('Mobility ($\mu$) in Kmph', fontsize = 12)
plt.ylabel('Spectral Efficiency (bps/Hz)', fontsize = 12)
plt.xticks([r + 4*barWidth for r in range(len(horis))],
        ['3', '30', '120'])
# plt.ylim(-4, 0)
plt.legend()
plt.show()


# plt.plot(horis,se_stem_512, label=r'STEM, $\gamma = 1/4$', marker = 'o', ms = 7, c = 'r', linewidth=2)
# plt.plot(horis,se_tran_512, label=r'TRANSFORMER, $\gamma = 1/4$', marker = 's', ms = 6, c = 'darkviolet', linewidth=2)
# plt.plot(horis,se_lstm_512, label=r'LSTM, $\gamma = 1/4$', marker = '^', ms = 7, c = 'g', linewidth=2)
# plt.plot(horis,se_rnn_512, label=r'RNN, $\gamma = 1/4$', marker = '*', ms = 8, c = 'b', linewidth=2)


# plt.plot(horis,se_stem_256, label=r'STEM, $\gamma = 1/8$', marker = 'o', ms = 7, c = 'tab:orange', linewidth=2)
# plt.plot(horis,se_tran_256, label='TRANSFORMER, $\gamma = 1/8$', marker = 's', ms = 6, c = 'm', linewidth=2)
# plt.plot(horis,se_lstm_256, label=r'LSTM, $\gamma = 1/8$', marker = '^', ms = 7, c = 'tab:green', linewidth=2)
# plt.plot(horis,se_rnn_256, label=r'RNN, $\gamma = 1/8$', marker = '*', ms = 8, c = 'tab:blue', linewidth=2)


# plt.plot(horis,se_stem_128, label=r'STEM, $\gamma = 1/16$', marker = 'o', ms = 7, c = 'y', linewidth=2)
# plt.plot(horis,se_tran_128, label='TRANSFORMER, $\gamma = 1/16$', marker = 's', ms = 6, c = 'tab:pink', linewidth=2)
# plt.plot(horis,se_lstm_128, label=r'LSTM, $\gamma = 1/16$', marker = '^', ms = 7, c = 'mediumseagreen', linewidth=2)
# plt.plot(horis,se_rnn_128, label=r'RNN, $\gamma = 1/16$', marker = '*', ms = 8, c = 'xkcd:sky blue', linewidth=2)

# plt.xlabel(r'Mobility ($\mu$) in Kmph', fontsize = 12)
# plt.ylabel("Spectral Efficiency (bps/Hz)", fontsize = 12)
# plt.legend()
# plt.title('$N_{t}=32$, $N_{c}=32$, $\mu = 120 Kmph$')
# plt.grid(True)
# plt.show()

In [32]:
rmse_stem_512 = [1.5832, 2.6189, 4.1774, 5.4101]
rmse_stem_256 = [1.9206, 3.0388, 5.2129, 5.6265]
rmse_stem_128 = [2.5506, 3.3334, 4.4479, 6.0265]

rmse_lstm_128 = [27.193055741, 26.6125009050, 26.7770670050, 27.7291775343]
rmse_lstm_256 = [24.729510545, 24.9839926950, 24.7412368050, 25.2967482805]
rmse_lstm_512 = [24.440444529, 23.7080699205, 23.9851315617, 24.3868720531]

rmse_rnn_128 = [25.2789484262, 24.2054104208, 24.9343266487, 24.8131954073]
rmse_rnn_256 = [23.0762034058, 22.9603829382, 23.6214694383, 24.7122474310]
rmse_rnn_512 = [22.4404445290, 22.7080699205, 22.9851315617, 24.3868720531]

rmse_tran_128 = [22.2789484262, 21.2054104208, 22.9343266487, 22.8131954073]
rmse_tran_256 = [21.0762034058, 21.9603829382, 22.6214694383, 22.7122474310]
rmse_tran_512 = [20.166, 21.7080699205, 21.9851315617, 22.3868720531]

barWidth = 0.07
fig = plt.subplots(figsize =(12, 8))
horis = [3, 5, 7, 9]

br1 = np.arange(len(horis))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]
br4 = [x + barWidth for x in br3]
br5 = [x + barWidth for x in br4]
br6 = [x + barWidth for x in br5]
br7 = [x + barWidth for x in br6]
br8 = [x + barWidth for x in br7]
br9 = [x + barWidth for x in br8]
br10 = [x + barWidth for x in br9]
br11 = [x + barWidth for x in br10]
br12 = [x + barWidth for x in br11]

plt.bar(br1, rmse_stem_128, color ='y', width = barWidth,
        edgecolor ='grey', label ='STEM-128')
plt.bar(br2, rmse_stem_256, color ='tab:orange', width = barWidth,
        edgecolor ='grey', label ='STEM-256')
plt.bar(br3, rmse_stem_512, color ='r', width = barWidth,
        edgecolor ='grey', label ='STEM-512')

plt.bar(br4, rmse_tran_128, color ='tab:pink', width = barWidth,
        edgecolor ='grey', label ='TRANSFORMER-512')
plt.bar(br5, rmse_tran_256, color ='m', width = barWidth,
        edgecolor ='grey', label ='TRANSFORMER-256')
plt.bar(br6, rmse_tran_512, color ='darkviolet', width = barWidth,
        edgecolor ='grey', label ='TRANSFORMER-128')

plt.bar(br7, rmse_rnn_128, color ='mediumseagreen', width = barWidth,
        edgecolor ='grey', label ='LSTM-128')
plt.bar(br8, rmse_rnn_256, color ='tab:green', width = barWidth,
        edgecolor ='grey', label ='LSTM-256')
plt.bar(br9, rmse_rnn_512, color ='g', width = barWidth,
        edgecolor ='grey', label ='LSTM-512')


plt.bar(br10, rmse_lstm_128, color ='xkcd:sky blue', width = barWidth,
        edgecolor ='grey', label ='RNN-128')
plt.bar(br11, rmse_lstm_256, color ='tab:blue', width = barWidth,
        edgecolor ='grey', label ='RNN-256')
plt.bar(br12, rmse_lstm_512, color ='b', width = barWidth,
        edgecolor ='grey', label ='RNN-512')
 
# Adding Xticks
plt.xlabel('Horizon', fontsize = 12)
plt.ylabel('RMSE after Compressed Channel Prediction', fontsize = 12)
plt.xticks([r + 4*barWidth for r in range(len(horis))],
        ['3', '5', '7', '9'])
# plt.ylim(-4, 0)
plt.legend()
plt.show()

In [22]:
train_loss_stem = [0.2698, 0.0473, 0.0117, 0.0086, 0.0059, 0.0053, 0.0047, 0.0043, 0.0040, 0.0034, 0.0033, 0.0031, 0.0030, 0.0029, 0.0027, 0.0026, 0.0026, 0.0025, 0.0025, 0.0024, 0.0024, 0.0023, 0.0023, 0.0023, 0.0022, 0.0022, 0.0022, 0.0022, 0.0022, 0.0022, 0.0022, 0.0021, 0.0021, 0.0021, 0.0021, 0.0021, 0.0021, 0.0021, 0.0021, 0.0021, 0.0021]
train_rmse_stem = [0.2983, 0.1806, 0.1396, 0.1140, 0.1230, 0.0893, 0.0834, 0.0800, 0.0763, 0.0759, 0.0712, 0.0690, 0.0672, 0.0654, 0.0651, 0.0634, 0.0634, 0.0621, 0.0612, 0.0614, 0.0605, 0.0600, 0.0597, 0.0593, 0.0591, 0.0590, 0.0586, 0.0585, 0.0584, 0.0581, 0.0580, 0.0579, 0.0579, 0.0577, 0.0576, 0.0575, 0.0575, 0.0575, 0.0574, 0.0573, 0.0573]
# plt.plot(train_rmse_stem)
# plt.plot(avg_loss[0:40])
# plt.xlabel('epochs',fontweight ='bold', fontsize = 12)
# plt.ylabel('RMSE loss during training',fontweight ='bold', fontsize = 12)
# plt.show()


fig = plt.figure()
ax = fig.add_subplot(111)
lns1 = ax.plot(train_rmse_stem, '-', label = 'STEM', color ='y', linewidth=2)
ax2 = ax.twinx()
lns2 = ax2.plot(avg_loss_rnn[0:40], '-', label = 'LSTM', color ='mediumseagreen', linewidth=2)
lns3 = ax2.plot(avg_loss_lstm[0:40], '-', label = 'RNN', color ='xkcd:sky blue', linewidth=2)
lns4 = ax2.plot(avg_loss_tran[0:40], '-', label = 'Transformer', color ='tab:pink', linewidth=2)

ax.grid()
ax.set_xlabel('epochs', fontsize = 12)
ax.set_ylabel('RMSE loss during training for STEM', fontsize = 12)
ax2.set_ylabel('RMSE loss during training for Transformer/LSTM/RNN', fontsize = 12)
lns = lns1+lns4+lns2+lns3
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc=0)
plt.show()